<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/video-translation-uk-ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U moviepy
!pip install -q -U whisperx
!pip install -q -U deepl
!pip install -q -U TTS
!pip install -q -U librosa

ERROR: Could not find a version that satisfies the requirement whisperx (from versions: none)
ERROR: No matching distribution found for whisperx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.7/908.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.1 MB/s eta 0:00:00
     ━━━

In [ ]:
import gradio as gr
import os
import moviepy.editor as mp
import whisperx
import deepl
import coqui_tts
import librosa

# Define function to extract audio from video
def extract_audio(video_path):
    clip = mp.VideoFileClip(video_path)
    audio_path = os.path.splitext(video_path)[0] + ".wav"
    clip.audio.write_audiofile(audio_path)
    return audio_path

# Define function to perform speech diarization
def speech_diarization(audio_path):
    diarization = whisperx.Diarization(audio_path)
    return diarization.transcription

# Define function to perform text translation
def text_translation(text, target_language):
    translation = deepl.translate(text, target_language)
    return translation

# Define function to perform voice cloning
def voice_cloning(text, speaker_model_path):
    tts = coqui_tts.TTS(speaker_model_path)
    audio = tts.get_audio(text)
    return audio

# Define function to adjust voice pace
def adjust_voice_pace(audio_path, target_duration):
    y, sr = librosa.load(audio_path, sr=None)
    duration = librosa.get_duration(y=y, sr=sr)
    tempo = duration / target_duration
    y_stretched = librosa.effects.time_stretch(y, tempo)
    librosa.output.write_wav(audio_path, y_stretched, sr)
    return audio_path

# Define function to perform video translation
def video_translation(video_path, target_language, speaker_model_path):
    # Extract audio from video
    audio_path = extract_audio(video_path)
    # Perform speech diarization
    transcription = speech_diarization(audio_path)
    # Perform text translation
    translated_transcription = []
    for text in transcription:
        translated_text = text_translation(text, target_language)
        translated_transcription.append(translated_text)
    # Perform voice cloning
    cloned_audio_paths = []
    for i, text in enumerate(translated_transcription):
        speaker_model_path_i = speaker_model_path[i] if len(speaker_model_path) > i else speaker_model_path[-1]
        cloned_audio_path = voice_cloning(text, speaker_model_path_i)
        cloned_audio_paths.append(cloned_audio_path)
    # Adjust voice pace
    target_duration = mp.VideoFileClip(video_path).duration
    for i, audio_path in enumerate(cloned_audio_paths):
        cloned_audio_paths[i] = adjust_voice_pace(audio_path, target_duration)
    # Combine video with new audio
    video = mp.VideoFileClip(video_path)
    audio_clips = [mp.AudioFileClip(audio_path) for audio_path in cloned_audio_paths]
    audio = mp.concatenate_audioclips(audio_clips)
    video_with_new_audio = video.set_audio(audio)
    # Save video with new audio
    video_with_new_audio_path = os.path.splitext(video_path)[0] + "_" + target_language + ".mp4"
    video_with_new_audio.write_videofile(video_with_new_audio_path)
    return video_with_new_audio_path


In [ ]:
def translate_video(video_path, target_language, speaker_model_path):
    video_with_new_audio_path = video_translation(video_path, target_language, speaker_model_path)
    return gr.outputs.Video(video_with_new_audio_path)
inputs = [
    gr.inputs.Video(type="file", label="Select a video file"),
    gr.inputs.Dropdown(["uk", "ru"], label="Select target language"),
    gr.inputs.Text(label="Speaker model path (optional)", default="")
]
outputs = gr.outputs.Video(label="Translated video")
gr.Interface(fn=translate_video, inputs=inputs, outputs=outputs, title="YouTube Video Translation Program").launch()

